In [ ]:
# same data as first cell: full calendar with missing days = NaN
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers

# same data as first cell: full calendar with missing days = NaN
data_path = Path("../data/crobex_history.csv")
df = pd.read_csv(data_path, sep=";", quotechar='"', decimal=",", parse_dates=["date"])
df = df.sort_values("date")

date_range = pd.date_range(df["date"].min(), df["date"].max(), freq="D")
plot_df = pd.DataFrame({"date": date_range})
plot_df = plot_df.merge(df[["date", "last_value"]], on="date", how="left")
observed = plot_df["last_value"].copy()

# train RNN on trading days only: predict next trading day from seq_len previous trading days
ts = df["last_value"].dropna().values.astype(np.float32).reshape(-1, 1)
scaler = MinMaxScaler()
ts_scaled = scaler.fit_transform(ts)
seq_len = 20
X = np.array([ts_scaled[i : i + seq_len] for i in range(len(ts_scaled) - seq_len)])
y = np.array([ts_scaled[i + seq_len, 0] for i in range(len(ts_scaled) - seq_len)])

model = keras.Sequential([
        layers.Input(shape=(seq_len, 1)),
    layers.SimpleRNN(16),
    layers.Dense(1),
])
model.compile(optimizer="adam", loss="mse")
model.fit(X, y, epochs=30, batch_size=32, verbose=0)

# imputation: use only observed values in the window; one prediction per gap (no autoregression over calendar)
# so we match training (next value from last seq_len observed) and avoid error compounding.
obs_mask = ~np.isnan(observed.values)
obs_indices = np.where(obs_mask)[0]  # calendar indices where we have data
vals = observed.values.astype(np.float64).copy()

def get_observed_window_before(calendar_idx):
    """Last seq_len observed values ending before calendar_idx. Padding with earliest available if needed."""
    before = obs_indices[obs_indices < calendar_idx]
    if len(before) == 0:
        return None
    # last obs index (in calendar)
    last_obs = before[-1]
    # indices of last seq_len observed up to and including last_obs
    up_to = obs_indices[obs_indices <= last_obs][-seq_len:]
    if len(up_to) < seq_len:
        pad_len = seq_len - len(up_to)
        first_obs_val = vals[up_to[0]]
        window = np.concatenate([np.full(pad_len, first_obs_val), vals[up_to].astype(np.float64)])
    else:
        window = vals[up_to].astype(np.float64)
    return window

# find contiguous gaps and fill each gap with a single RNN prediction (from last seq_len observed)
i = 0
while i < len(vals):
    if not np.isnan(vals[i]):
        i += 1
        continue
    gap_start = i
    while i < len(vals) and np.isnan(vals[i]):
        i += 1
    gap_end = i
    window = get_observed_window_before(gap_start)
    if window is None or len(window) != seq_len:
        # not enough history: forward-fill from next observed (or from later in series)
        fill_val = vals[gap_end] if gap_end < len(vals) and not np.isnan(vals[gap_end]) else np.nan
        if np.isnan(fill_val):
            # no next observed; backward-fill from last observed before gap
            prev_obs = obs_indices[obs_indices < gap_start]
            fill_val = vals[prev_obs[-1]] if len(prev_obs) else np.nan
        if not np.isnan(fill_val):
            vals[gap_start:gap_end] = fill_val
        continue
    window_scaled = scaler.transform(window.reshape(-1, 1))
    pred = model.predict(window_scaled.reshape(1, seq_len, 1), verbose=0)
    fill_val = scaler.inverse_transform(pred)[0, 0]
    vals[gap_start:gap_end] = fill_val

imputed = pd.Series(vals, index=plot_df.index)


In [11]:
# same chart design: observed (gaps) + rnn-imputed (filled)
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=plot_df["date"],
        y=imputed,
        mode="lines",
        name="rnn imputed",
        line=dict(color="#ef553b", dash="solid"),
        connectgaps=True,
    )
)
fig.add_trace(
    go.Scatter(
        x=plot_df["date"],
        y=observed,
        mode="lines",
        name="observed (with gaps)",
        line=dict(color="#636efa", dash="solid"),
        connectgaps=False,
    )
)
fig.update_layout(
    hovermode="x unified",
    title=None,
    xaxis_title=None,
    yaxis_title=None,
    yaxis=dict(ticksuffix=" €"),
    showlegend=True,
)
fig.update_traces(connectgaps=False, selector=dict(name="observed (with gaps)"))

fig.show()